In [1]:
import pandas as pd
import numpy as np
import config
import datetime
import sys
from sqlalchemy import text
print(sys.version)
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]
3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=10
year=21

In [3]:
def get_ps(year, specialite):
    sql=f"""
select sp.specialite_id, s.label_long as label, ds.annee as year, ps.id as p_id, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join date_source ds on ds.id=tds.date_source_id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
where sp.specialite_id={specialite}
and ds.annee={year}
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string).dropna()


In [4]:
def get_pa(year, specialite):
    sql=f"""
select sp.specialite_id, ds.annee as year, pa.id as p_id, an.lon, an.lat from personne_activite pa
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join date_source ds on ds.id=pands.date_source_id
where sp.specialite_id={specialite}
and pa.code_mode_exercice='L'
and ds.annee={year}
"""
    print(f"Quering PA for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string).dropna()

In [5]:
def get_pa_outer_ps(year, specialite):
    sql=f"""
with ps2 as (
    select ps.id, ps.key, an.lon, an.lat from ps
    join tarif t on t.ps_id = ps.id
    join tarif_date_source tds on tds.tarif_id=t.id
    join date_source ds on ds.id=tds.date_source_id
    join cabinet c on t.cabinet_id=c.id
    join adresse_raw ar on c.adresse_raw_id=ar.id
    join adresse_norm an on ar.adresse_norm_id=an.id
    join specialite_profession sp on sp.profession_id=t.profession_id
    --join specialite s on s.id=sp.specialite_id
    where sp.specialite_id={specialite}
    and ds.annee={year}
)
select sp.specialite_id, s.label_long as label, ds.annee as year, pa.id as p_id, coalesce(an.lon,ps2.lon) as lon, coalesce(an.lat,ps2.lat) as lat
from personne_activite pa
left outer join ps2 on ps2.key=pa.inpp
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join date_source ds on ds.id=pands.date_source_id
join specialite s on s.id=sp.specialite_id
where sp.specialite_id={specialite}
and pa.code_mode_exercice='L'
and ds.annee={year}
"""
    print(f"Quering PA-Outer-PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string).dropna()

In [6]:
def get_pa_ps(year, specialite):
    sql=f"""
select sp.specialite_id, ds.annee as year, pa.id as p_id, coalesce(an.lon,an2.lon) as lon, coalesce(an.lat,an2.lat) as lat from personne_activite pa
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join date_source ds on ds.id=pands.date_source_id
left outer join ps on ps.key=pa.inpp
join tarif t on t.ps_id=ps.id
join tarif_date_source tds on tds.tarif_id=t.id and tds.date_source_id=ds.id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an2 on ar.adresse_norm_id=an2.id
where sp.specialite_id={specialite}
and pa.code_mode_exercice='L'
and ds.annee={year}
"""
    print(f"Quering PA-PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string).dropna()

In [7]:
def get_rpps(year, specialite):
    sql=f"""
select personne.id p_id,an.lon,an.lat from personne
join diplome_obtenu dio on dio.personne_id = personne.id
join diplome d on d.id=dio.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join coord c on c.personne_id=personne.id
join adresse_norm an on c.adresse_norm_id=an.id
where sp.specialite_id={specialite}
and personne.date_maj >= '20{year}-01-01' and date_effet <= '20{year}-12-31'
"""
    print(f"Quering RPPS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string).dropna()

In [8]:
df2=get_pa(year,specialite)
df2

Quering PA for year 21 and specialite 10


,specialite_id,year,p_id,lon,lat
0,10,21,51080,3.016153,43.193501
1,10,21,51080,3.016153,43.193501
2,10,21,51080,3.016153,43.193501
6,10,21,51088,4.417738,44.011191
7,10,21,51088,4.318668,43.822473
...,...,...,...,...,...
186934,10,21,781240,-0.587016,44.829332
186935,10,21,781240,-0.587016,44.829332
186939,10,21,1539728,3.124607,50.311233
186940,10,21,1539728,3.124607,50.311233


In [9]:
nb_unique = len(df2.groupby(["p_id", "lon", "lat"]))
nb_unique

50966

In [10]:
nb = df2["p_id"].nunique()
nb

46845

In [11]:
ameli={1:6214,2:3859,3:2433,4:2021,5:4368,6:4295,7:2704,8:5518,9:36062,10:51488,11:4989,12:7777,13:842,14:115,15:1036,16:1971,17:1196,18:505,19:1449,20:704}
ameli

{1: 6214,
 2: 3859,
 3: 2433,
 4: 2021,
 5: 4368,
 6: 4295,
 7: 2704,
 8: 5518,
 9: 36062,
 10: 51488,
 11: 4989,
 12: 7777,
 13: 842,
 14: 115,
 15: 1036,
 16: 1971,
 17: 1196,
 18: 505,
 19: 1449,
 20: 704}

In [12]:
df = pd.DataFrame(columns=["specialite","label","year","ameli","nb_ps_cabinet","nb_ps","delta_ps","nb_pa_cabinet","nb_pa","delta_pa","nb_pa_ps_cabinet","nb_pa_ps","delta_pa_ps","nb_pa_outer_ps_cabinet","nb_pa_outer_ps","delta_pa_outer_ps","nb_rpps_cabinet","np_rpps","delta_rpps"])
for specialite in range(10,11):
    for yy in range(21,22): #datetime.date.today().year+1-2000):
        ps_df=get_ps(yy,specialite)
        label=ps_df["label"][0] if len(ps_df["label"])>0 else ""
        nb_unique_ps=len(ps_df.groupby(["p_id", "lon", "lat"]))
        nb_ps=ps_df["p_id"].nunique()
        pa_df=get_pa(yy,specialite)
        nb_unique_pa=len(pa_df.groupby(["p_id", "lon", "lat"]))
        nb_pa=pa_df["p_id"].nunique()
        pa_ps_df=get_pa_ps(yy,specialite)
        nb_unique_pa_ps=len(pa_ps_df.groupby(["p_id", "lon", "lat"]))
        nb_pa_ps=pa_ps_df["p_id"].nunique()
        pa_ops_df=get_pa_outer_ps(yy,specialite)
        nb_unique_pa_ops=len(pa_ops_df.groupby(["p_id", "lon", "lat"]))
        nb_pa_ops=pa_ops_df["p_id"].nunique()
        rpps_df=get_rpps(yy,specialite)
        nb_unique_rpps=len(rpps_df.groupby(["p_id", "lon", "lat"]))
        nb_rpps=rpps_df["p_id"].nunique()
        a=np.nan
        if yy==23 and specialite in ameli:
            a=ameli[specialite]
        df = pd.concat([pd.DataFrame([[specialite,label,yy,a,nb_unique_ps,nb_ps,(nb_ps-a)/a,nb_unique_pa,nb_pa,(nb_pa-a)/a,nb_unique_pa_ps,nb_pa_ps,(nb_pa_ps-a)/a,nb_unique_pa_ops,nb_pa_ops,(nb_pa_ops-a)/a,nb_unique_rpps,nb_rpps,(nb_rpps-a)/a]], columns=df.columns), df], ignore_index=True)
df=df.sort_values(by=['specialite',"year"])
df

Quering PS for year 21 and specialite 10
Quering PA for year 21 and specialite 10
Quering PA-PS for year 21 and specialite 10
Quering PA-Outer-PS for year 21 and specialite 10
Quering RPPS for year 21 and specialite 10


,specialite,label,year,ameli,nb_ps_cabinet,nb_ps,delta_ps,nb_pa_cabinet,nb_pa,delta_pa,nb_pa_ps_cabinet,nb_pa_ps,delta_pa_ps,nb_pa_outer_ps_cabinet,nb_pa_outer_ps,delta_pa_outer_ps,nb_rpps_cabinet,np_rpps,delta_rpps
0,10,Généraliste,21,NaN,58199,54811,NaN,50966,46845,NaN,51498,46621,NaN,58618,52392,NaN,13667,13667,NaN


In [13]:
df.to_excel("ps_stats.xlsx", index=False)
df.to_sql("ps_stats", config.connection_string, schema="apl", if_exists="replace", index=False)

1